In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
from array import array
import math


class Vector2D:
    __slots__ = ('__x', '__y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        return '{}({})'.format(self.__class__.__name__,
                               ', '.join(repr(f) for f in self))

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = abs(self), self.angle()
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'

        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def angle(self):
        return math.atan2(self.y, self.x)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


"""

>>> v1 = Vector2D(3, 4)
>>> print(v1.x, v1.y)
3.0 4.0
>>> x, y = v1
>>> x, y
(3.0, 4.0)
>>> v1
Vector2D(3.0, 4.0)
>>> v1_clone = eval(repr(v1))
>>> v1 == v1_clone
True
>>> print(v1)
(3.0, 4.0)
>>> abs(v1)
5.0
>>> bool(v1), bool(Vector2D(0, 0))
(True, False)
>>> octets = bytes(v1)
>>> octets
b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
>>> v1_restore = Vector2D.frombytes(octets)
>>> v1 == v1_restore
True
>>> format(v1)
'(3.0, 4.0)'
>>> format(v1, '.2f')
'(3.00, 4.00)'
>>> format(v1, '.3e')
'(3.000e+00, 4.000e+00)'
>>> format(v1, 'p')
'<5.0, 0.9272952180016122>'
>>> format(v1, '.3ep')
'<5.000e+00, 9.273e-01>'
>>> format(v1, '0.5fp')
'<5.00000, 0.92730>'
>>> v2 = Vector2D(3.1, 4.2)
>>> hash(v1), hash(v2)
(7, 384307168202284039)
>>> {v1, v2}
{Vector2D(3.1, 4.2), Vector2D(3.0, 4.0)}
>>> class ShortVector2D(Vector2D):
...     typecode = 'f'
>>> sv = ShortVector2D(1/11, 1/27)
>>> repr(sv)
'ShortVector2D(0.09090909090909091, 0.037037037037037035)'
>>> bytes(sv)
b'f\\x8c.\\xba=&\\xb4\\x17='
>>> sv.typecode
'f'
"""

doctest.testmod()

TestResults(failed=0, attempted=28)